# 제목1
## 제목 2
### 제목 3
#### 제목 4

- 불릿1
- 불릿2

1. 리스트1
2. 리스트2

*이탤릭*
**볼드**

하이
헬로우

하이

헬로우

하이  
이  
헬로우

# 

In [1]:
import pandas as pd
import numpy as np

import requests
import json

from datetime import datetime

import folium
from folium.plugins import MarkerCluster, MiniMap

In [2]:
type_dict = {
    '01':'약국',
    '02':'우체국',
    '03':'농협'
}
store_list = []
i = 1

while True:
    store_url = 'https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/stores/json'
    PARAMS = {'page':i}
    response = requests.get(url=store_url, params=PARAMS)
    response_dict = response.json()

    page_count = response_dict['totalPages']

    for store in response_dict['storeInfos']:
        result = []
        keys = ['name', 'addr', 'code', 'type', 'lat', 'lng']
        for key in keys:
            if key in store:
                result.append(store[key])
            else:
                result.append(None)

        store_list.append(result)
    
    if page_count == i:
        break
    else:
        i += 1


store_df = pd.DataFrame(store_list, columns=['name', 'address', 'code', 'vendor', 'lat', 'lng'])
store_df['vendor'] = store_df.vendor.map(type_dict)
store_df['code'] = store_df.code.map(str)

In [3]:
store_df.head()

,name,address,code,vendor,lat,lng
0,신신약국,충청남도 홍성군 갈산면 상촌로 12-1,34840559,약국,36.602986,126.548911
1,밝은온누리약국,대구광역시 달서구 달구벌대로 1736 1층 (두류동),37844571,약국,35.856678,128.555278
2,지애약국,충청남도 보령시 남대천로 58 (대천동),34833323,약국,36.348742,126.593534
3,고령쌍림우체국,경상북도 고령군 쌍림면 대가야로 608,5038300744,우체국,35.679196,128.242253
4,굿모닝약국,부산광역시 금정구 서동로 168 송월빌딩 1층 102호 (서동),21839824,약국,35.214448,129.104775


In [4]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26252 entries, 0 to 26251
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     26252 non-null  object 
 1   address  26252 non-null  object 
 2   code     26252 non-null  object 
 3   vendor   26252 non-null  object 
 4   lat      26247 non-null  float64
 5   lng      26247 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.2+ MB


In [5]:
sales_list = []
i = 1

while True:
    sales_url = 'https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/sales/json'
    PARAMS = {'page':i}
    response = requests.get(url = sales_url, params = PARAMS)
    response_dict = response.json()

    page_count = response_dict['totalPages']

    for sales in response_dict['sales']:
        result = []
        keys = ['code', 'stock_at', 'remain_stat', 'created_at']
        for key in keys:
            if key in sales:
                result.append(sales[key])
            else:
                result.append(None)
            
        sales_list.append(result)
    
    if page_count == i:
        break
    else:
        i += 1

sales_df = pd.DataFrame(sales_list, columns = ['code', 'stock_at', 'remain_status', 'created_at'])
sales_df['code'] = sales_df.code.map(str)

In [6]:
sales_df.head()

,code,stock_at,remain_status,created_at
0,31837492,2020/04/11 09:30:00,break,2020/04/12 17:55:00
1,21836841,2020/04/11 09:08:00,break,2020/04/12 17:55:00
2,37839128,2020/04/11 11:46:00,break,2020/04/12 17:55:00
3,11831049,2020/04/11 11:54:00,plenty,2020/04/12 17:55:00
4,21839271,2020/04/12 09:20:00,some,2020/04/12 17:55:00


In [7]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25724 entries, 0 to 25723
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   code           25724 non-null  object
 1   stock_at       25676 non-null  object
 2   remain_status  25676 non-null  object
 3   created_at     25724 non-null  object
dtypes: object(4)
memory usage: 804.0+ KB


In [8]:
mask_df = pd.merge(sales_df, store_df, how = 'inner', on='code').drop_duplicates()
mask_df.head()

,code,stock_at,remain_status,created_at,name,address,vendor,lat,lng
0,31837492,2020/04/11 09:30:00,break,2020/04/12 17:55:00,현대약국,"경기도 파주시 시청로 209 (아동동, 호수빌딩)",약국,37.770944,126.775845
1,21836841,2020/04/11 09:08:00,break,2020/04/12 17:55:00,덕인약국,부산광역시 해운대구 우동2로 16 (우동),약국,35.166153,129.160501
2,37839128,2020/04/11 11:46:00,break,2020/04/12 17:55:00,민다약국,경상북도 고령군 대가야읍 왕릉로 105-1,약국,35.729189,128.267101
3,11831049,2020/04/11 11:54:00,plenty,2020/04/12 17:55:00,국일약국,서울특별시 종로구 종로16길 12 (관철동),약국,37.569560,126.988520
4,21839271,2020/04/12 09:20:00,some,2020/04/12 17:55:00,진하약국,부산광역시 기장군 일광면 일광로 609,약국,35.296527,129.258738


In [9]:
draw_df = mask_df[['name', 'lat', 'lng', 'remain_status']]
draw_df.head()

,name,lat,lng,remain_status
0,현대약국,37.770944,126.775845,break
1,덕인약국,35.166153,129.160501,break
2,민다약국,35.729189,128.267101,break
3,국일약국,37.569560,126.988520,plenty
4,진하약국,35.296527,129.258738,some


In [10]:
draw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25673 entries, 0 to 25672
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           25673 non-null  object 
 1   lat            25668 non-null  float64
 2   lng            25668 non-null  float64
 3   remain_status  25631 non-null  object 
dtypes: float64(2), object(2)
memory usage: 1002.9+ KB


In [11]:
# lat와 lng의 값이 없으면 지도에서 표시할 수 없기 떄문에 null을 제외
draw_df_clean = draw_df[draw_df['remain_status'] != 'break'].dropna()
draw_df_clean.shape

(16714, 4)

In [12]:
color_dict = {
    'plenty': 'green',
    'some': 'orange',
    'few': 'red',
    'empty': 'gray',
    'no_data': 'purple'
}

label_dict = {
    'plenty': '100개 이상',
    'some': '30개 이상',
    'few': '30개 미만',
    'empty': '재고 없음',
    'no_data': '정보 없음'
}

In [13]:
mask_map = folium.Map((36.586466, 128.186711), zoom_start = 7)
mc = MarkerCluster()

names = list(draw_df_clean['name'])
latitudes = list(draw_df_clean['lat'])
longitudes = list(draw_df_clean['lng'])
status = list(draw_df_clean['remain_status'])

for i in range(len(names)):
    mc.add_child(folium.Marker(
        location = [latitudes[i], longitudes[i]], 
        icon = folium.Icon(color = color_dict[status[i]]), 
        # popup = names[i] + ' ' + label_dict[status[i]])
        popup = f'{names[i]} ({label_dict[status[i]]})')
    )
    mask_map.add_child(mc)

In [14]:
mask_map.save("html/mask_map.html")